## Import Liberaries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import datasets ,layers,models
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import kagglehub
import os


## Import Data

In [2]:
path = kagglehub.dataset_download("misrakahmed/vegetable-image-dataset")
path

100%|██████████| 534M/534M [00:25<00:00, 21.8MB/s]

Extracting files...


'/root/.cache/kagglehub/datasets/misrakahmed/vegetable-image-dataset/versions/1'

In [3]:
data = path
files = os.listdir(data)
files

['Vegetable Images']

In [4]:
data_dir  = os.path.join(data,'Vegetable Images')
Vegetable_folders = os.listdir(data_dir)
Vegetable_folders

['validation', 'train', 'test']

In [ ]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')

### Apply ImageDataGenerator with Augmentation:

In [ ]:
# Define the data generator with augmentation
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize the image pixels to [0, 1]
    rotation_range=40,  # Randomly rotate the image by up to 40 degrees
    width_shift_range=0.2,  # Randomly shift the image horizontally
    height_shift_range=0.2,  # Randomly shift the image vertically
    shear_range=0.2,  # Apply shear transformations to the image
    zoom_range=0.2,  # Randomly zoom in or out of the image
    horizontal_flip=True,  # Randomly flip the image horizontally
    fill_mode='nearest'  # How to fill in missing pixels after transformations
)

In [21]:


train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

val_data = datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)


Found 15000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.


## CNN model

In [6]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(15, activation='softmax')  # 15 classes
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 15)                  │           1,935 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,306,575 (12.61 MB)

 Trainable params: 3,306,575 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(
    optimizer='adamax',
    loss='categorical_crossentropy',  # Use this for one-hot encoded labels
    metrics=['accuracy']
)


In [15]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=25,
    batch_size=32
)


Epoch 1/25
469/469 ━━━━━━━━━━━━━━━━━━━━ 90s 191ms/step - accuracy: 0.9612 - loss: 0.1228 - val_accuracy: 0.9760 - val_loss: 0.0781
Epoch 2/25
469/469 ━━━━━━━━━━━━━━━━━━━━ 82s 174ms/step - accuracy: 0.9613 - loss: 0.1306 - val_accuracy: 0.9790 - val_loss: 0.0777
Epoch 3/25
469/469 ━━━━━━━━━━━━━━━━━━━━ 83s 177ms/step - accuracy: 0.9624 - loss: 0.1157 - val_accuracy: 0.9733 - val_loss: 0.0908
Epoch 4/25
469/469 ━━━━━━━━━━━━━━━━━━━━ 81s 173ms/step - accuracy: 0.9621 - loss: 0.1233 - val_accuracy: 0.9750 - val_loss: 0.0767
Epoch 5/25
469/469 ━━━━━━━━━━━━━━━━━━━━ 84s 178ms/step - accuracy: 0.9635 - loss: 0.1233 - val_accuracy: 0.9773 - val_loss: 0.0728
Epoch 6/25
469/469 ━━━━━━━━━━━━━━━━━━━━ 82s 175ms/step - accuracy: 0.9634 - loss: 0.1186 - val_accuracy: 0.9753 - val_loss: 0.0830
Epoch 7/25
469/469 ━━━━━━━━━━━━━━━━━━━━ 83s 177ms/step - accuracy: 0.9612 - loss: 0.1153 - val_accuracy: 0.9777 - val_loss: 0.0721
Epoch 8/25
469/469 ━━━━━━━━━━━━━━━━━━━━ 82s 174ms/step - accuracy: 0.9617 - loss: 0

In [20]:
model.evaluate(train_data)

469/469 ━━━━━━━━━━━━━━━━━━━━ 66s 141ms/step - accuracy: 0.9876 - loss: 0.0424


[0.04115564376115799, 0.9878000020980835]

In [19]:
model.evaluate(val_data)

94/94 ━━━━━━━━━━━━━━━━━━━━ 13s 138ms/step - accuracy: 0.9794 - loss: 0.0632


[0.0656798705458641, 0.9803333282470703]